In [1]:
# %%capture
import os 
os.environ['CUDA_VISIBLE_DEVICES']='5'
os.environ['HF_ENDPOINT']='https://hf-mirror.com'
os.environ['HF_HOME']="/home/test/Workspace/znchen/tianxing_project/share/huggingface"
import torch
import argparse
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path

import requests, json
from PIL import Image
from io import BytesIO
from transformers import TextStreamer

from detector import Detector
from retriever import ClipRetriever

def load_image(image_file):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

args = argparse.Namespace(
    model_path="Hoar012/RAP-LLaVA-13b",
    model_base=None,
    device="cuda",
    conv_mode=None,
    temperature=0.2,
    max_new_tokens=512,
    load_8bit=False,
    load_4bit=False,
    debug=False,
    index_path=None,
    retrieval=True
)
# Model
disable_torch_init()

model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name, args.load_8bit, args.load_4bit, device=args.device)

if "phi" in model_name.lower():
    conv_mode = "phi3_instruct"
elif "v1" in model_name.lower():
    conv_mode = "llava_v1"    
else:
    conv_mode = "llava_v0"

if args.conv_mode is not None and conv_mode != args.conv_mode:
    print('[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}'.format(conv_mode, args.conv_mode, args.conv_mode))
else:
    args.conv_mode = conv_mode

/home/test/Workspace/znchen/tianxing_project/anaconda3/envs/rap/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some weights of the model checkpoint at Hoar012/RAP-LLaVA-13b were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.class_embedding', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc2.w

In [ ]:
args.image_file="/home/test/Workspace/znchen/tianxing_project/share/NJU/NLP/RAP_Plus/tmp/frame_0.png"
args.retrieval=True
args.database="/home/test/Workspace/znchen/tianxing_project/share/NJU/NLP/RAP_Plus/example_database"
args.topK=2
#-------------------------------------#
if args.index_path is None:
    retriever = ClipRetriever(data_dir = args.database, index_path = args.index_path, create_index = True)
else:
    retriever = ClipRetriever(data_dir = args.database, index_path = args.index_path, create_index = False)
conv = conv_templates[args.conv_mode].copy()
if "mpt" in model_name.lower():
    roles = ('user', 'assistant')
else:
    roles = conv.roles

if args.retrieval:    
    detector = Detector()
    with open(f"{args.database}/database.json", "r") as f:
        database = json.load(f)
    
    # Set interested classes
    all_category = []
    for concept in database["concept_dict"]:
        cat = database["concept_dict"][concept]["category"]
        if cat not in all_category:
            all_category.append(cat)
    detector.model.set_classes(all_category)
      
image = load_image(args.image_file)
image_sizes = [image.size]
images = [image]
image_tensor = None



/home/test/Workspace/znchen/tianxing_project/anaconda3/envs/rap/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: RAP-MLLM-main/clip-vit-large-patch14-336 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
inp="Describe this frame!"

if image is not None:
    # first message
    if args.retrieval:
        crops = detector.detect_and_crop(image)
        extra_info, rag_images = retriever.retrieve(database, inp, queries = crops, topK = args.topK)
        
        for i, ret_path in enumerate(rag_images):
            img = load_image(ret_path)
            image_sizes.append(img.size)
            images.append(img)
            print(ret_path)
        
        inp = DEFAULT_IMAGE_TOKEN + f"\n[{extra_info}]" + inp
    else:
        inp = DEFAULT_IMAGE_TOKEN + "\n" + inp
        
    conv.append_message(conv.roles[0], inp)
    image = None
else:
    # later messages
    conv.append_message(conv.roles[0], inp)
conv.append_message(conv.roles[1], None)

# Similar operation in model_worker.py
if image_tensor is None:
    image_tensor = process_images(images, image_processor, model.config)
if type(image_tensor) is list:
    image_tensor = [image.to(model.device, dtype=torch.float16) for image in image_tensor]
else:
    image_tensor = image_tensor.to(model.device, dtype=torch.float16)
prompt = conv.get_prompt()
# print(prompt)
input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
print(f"{roles[1]}: ", end="")
with torch.inference_mode():
    output_ids = model.generate(
        input_ids,
        images=image_tensor,
        image_sizes=image_sizes,
        do_sample=True if args.temperature > 0 else False,
        temperature=args.temperature,
        max_new_tokens=args.max_new_tokens,
        streamer=streamer,
        use_cache=True)

outputs = tokenizer.decode(output_ids[0]).strip()
conv.messages[-1][-1] = outputs

if args.debug:
    print("\n", {"prompt": prompt, "outputs": outputs}, "\n")



0: 640x384 2 persons, 128.9ms
Speed: 3.0ms preprocess, 128.9ms inference, 24.6ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/detect/predict


NameError: name 'retriever' is not defined